In [10]:
import pandas as pd

# Load PM10 data
pm10_2021 = pd.read_csv('PM10_2021.csv')

# Convert Date column to datetime
pm10_2021['Date'] = pd.to_datetime(pm10_2021['Date'], errors='coerce')

# Drop missing values in Date or PM10
pm10_2021 = pm10_2021.dropna(subset=['Date', 'Daily Mean PM10 Concentration'])

# Clean County column
pm10_2021['County'] = pm10_2021['County'].str.strip().str.title()

# Group by County to get annual average PM10
pm10_avg = (
    pm10_2021.groupby('County')['Daily Mean PM10 Concentration']
    .mean()
    .reset_index()
    .rename(columns={'Daily Mean PM10 Concentration': 'Avg_PM10'})
)


In [11]:
# Load TB data
tb_2021 = pd.read_csv('2021 Illinois Tuberculosis Case Rates per 100,000 Population.csv')

# Clean County column
tb_2021['County'] = tb_2021['County'].str.strip().str.title()

# Rename TB rate column
tb_2021 = tb_2021.rename(columns={
    'Case Rate per 100,000 Population': 'TB_Rate'
})


In [13]:
print("PM10 Counties:\n", sorted(pm10_avg['County'].unique()))
print("\nTB Counties:\n", sorted(tb_2021['County'].unique()))


PM10 Counties:
 ['Cook', 'Madison']

TB Counties:
 ['Adams', 'Alexander', 'Bond', 'Boone', 'Brown', 'Bureau', 'Calhoun', 'Carroll', 'Cass', 'Champaign', 'Chicago', 'Christian', 'Clark', 'Clay', 'Clinton', 'Coles', 'Cook', 'Crawford', 'Cumberland', 'De Witt', 'Dekalb', 'Douglas', 'Dupage', 'Edgar', 'Edwards', 'Effingham', 'Fayette', 'Ford', 'Franklin', 'Fulton', 'Gallatin', 'Greene', 'Grundy', 'Hamilton', 'Hancock', 'Hardin', 'Henderson', 'Henry', 'Iroquois', 'Jackson', 'Jasper', 'Jefferson', 'Jersey', 'Jo Daviess', 'Johnson', 'Kane', 'Kankakee', 'Kendall', 'Knox', 'Lake', 'Lasalle', 'Lawrence', 'Lee', 'Livingston', 'Logan', 'Macon', 'Macoupin', 'Madison', 'Marion', 'Marshall', 'Mason', 'Massac', 'Mcdonough', 'Mchenry', 'Mclean', 'Menard', 'Mercer', 'Monroe', 'Montgomery', 'Morgan', 'Moultrie', 'Ogle', 'Peoria', 'Perry', 'Piatt', 'Pike', 'Pope', 'Pulaski', 'Putnam', 'Randolph', 'Richland', 'Rock Island', 'Saline', 'Sangamon', 'Schuyler', 'Scott', 'Shelby', 'St. Clair', 'Stark', 'Stephen